In [1]:
# default_exp metaflow

In [80]:
# export
import os
from pathlib import Path
from typing import Iterable

from fastcore.script import call_parse
from nbdev.export import Config, find_default_export, nbglob, read_nb
from sciflow.params import params_as_dict
from sciflow.parse_module import FuncDetails, extract_steps

# Sciflow Notebook to MetaFlow Flow

> Converts from a `sciflow` format notebook to a `metaflow` flow. 

Supported features:

* Linear/sequential DAGs
* Simple `Parameters`

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
nb_path = Path(os.path.join("test", "test_export.ipynb"))
nb = read_nb(nb_path)
module_name = find_default_export(nb["cells"])
test_module = os.path.join(Config().path("lib_path"), f"{module_name}.py")

In [5]:
# export


def titleize(name):
    return name.title().replace("_", "")

In [6]:
assert titleize("snake_case") == "SnakeCase"

In [7]:
# export


def rename_steps_for_metaflow(steps):
    for i, step in enumerate(steps):
        if i == 0:
            step.name = "start"
        if i == len(steps) - 1:
            step.name = "end"

In [8]:
steps = extract_steps(test_module)

In [9]:
no_steps = extract_steps(os.path.join(Config().path("lib_path"), f"_nbdev.py"))
assert len(no_steps) == 0

In [10]:
assert ["first", "preprocess", "train", "last"] == [step.name for step in steps]
rename_steps_for_metaflow(steps)
assert ["start", "preprocess", "train", "end"] == [step.name for step in steps]

In [11]:
# export


def indent_multiline(multiline_text, indent=1):
    lines = multiline_text.strip().split("\n")
    spaces = "".join(["    " for _ in range(indent)])
    for i in range(len(lines)):
        prefix = spaces if i > 0 else spaces + '"""'
        lines[i] = prefix + lines[i]
    return "\n".join(lines) + '"""'

In [12]:
text = """
Some text
:param param: text
"""
assert '    """Some text\n    :param param: text"""' == indent_multiline(text)

In [134]:
# export


def nb_to_metaflow(nb_path: Path, flow_path: Path, silent=True):
    nb = read_nb(nb_path)
    lib_name = Config().lib_name
    module_name = find_default_export(nb["cells"])
    nb_name = os.path.basename(nb_path)
    if not module_name:
        print(f"Skipping {nb_name} - no export module")
        return
    exported_module = os.path.join(Config().path("lib_path"), f"{module_name}.py")
    steps = extract_steps(exported_module)
    if len(steps) == 0:
        if not silent:
            print(f"Skipping {nb_name} - no steps")
        return
    orig_step_names = [step.name for step in steps]
    if len(steps) == 1:
        steps.append(FuncDetails("end", None, None, False, "pass"))
    params = params_as_dict(nb_path)
    if len(params) == 0:
        print(f"No params cell found for: {os.path.basename(nb_path)}")
    flow_class_name = f"{titleize(module_name)}Flow"
    rename_steps_for_metaflow(steps)
    write_module_to_file(
        flow_path,
        flow_class_name,
        lib_name,
        module_name,
        orig_step_names,
        steps,
        params,
    )
    if not silent:
        print(f"Wrote {flow_class_name} to: {os.path.basename(flow_path)}")

In [169]:
",".join(["_" + a for a in steps[2].args.split(",")])

'_input_path,_model_path'

In [172]:
#  export


def write_module_to_file(
    flow_path: Path,
    flow_class_name: str,
    lib_name: str,
    module_name: str,
    orig_step_names: Iterable[str],
    steps: Iterable[FuncDetails],
    params: dict,
):
    if not os.path.exists(flow_path.parent):
        os.mkdir(flow_path.parent)
    with open(flow_path, "w") as flow_file:
        flow_file.write("# SCIFLOW GENERATED FILE - DO NOT EDIT\n")
        flow_file.write("from metaflow import FlowSpec, step, Parameter\n")
        flow_file.write(
            f"from {lib_name}.{module_name} import {', '.join(orig_step_names)}\n"
        )
        flow_file.write(
            f"from {lib_name}.{module_name} import {', '.join(params.keys())}\n"
        )
        flow_file.write(f"\n\nclass {flow_class_name}(FlowSpec):\n")
        single_indent = "    "
        for param in params.keys():
            flow_file.write(
                f"{single_indent}{param} = Parameter('{param}', default={param})\n"
            )

        flow_file.write("\n")
        for i, step in enumerate(steps):
            flow_file.write(f"{single_indent}@step\n")
            flow_file.write(f"{single_indent}def {step.name}(self):\n")
            if step.docstring:
                flow_file.write(f"{indent_multiline(step.docstring, 2)}\n")
            # Check for padded step
            if i < len(orig_step_names):
                flow_step_args = ",".join(["self." + a for a in step.args.split(",")])
                flow_file.write(
                    f"{single_indent}{single_indent}{orig_step_names[i]}({flow_step_args})\n"
                )
            else:
                flow_file.write(f"{single_indent}{single_indent}pass\n")
            if i < len(steps) - 1:
                flow_file.write(
                    f"{single_indent}{single_indent}self.next(self.{steps[i+1].name})\n"
                )
            flow_file.write("\n")

        flow_file.write('if __name__ == "__main__":\n')
        flow_file.write(f"{single_indent}{flow_class_name}()")

In [154]:
# export


def get_module_name(nb_path):
    nb = read_nb(nb_path)
    module_name = find_default_export(nb["cells"])
    return module_name

In [155]:
flow_path = Path(
    os.path.join(Path(".").resolve(), "test", "flows", f"{get_module_name(nb_path)}.py")
)

In [156]:
flow_path

Path('/home/jovyan/git/sciflow/nbs/test/flows/test_export.py')

In [157]:
nb_to_metaflow(nb_path, flow_path, silent=False)

Wrote TestExportFlow to: test_export.py


# Ignore notebooks without Sciflow steps

In [158]:
nb_to_metaflow("packaging.ipynb", flow_path, silent=False)

Skipping packaging.ipynb - no steps


In [159]:
# export


def generate_flows(config: Config):
    flows_dir = config.path("flows_path")
    nb_paths = nbglob(recursive=True)
    for nb_path in nb_paths:
        flow_module_name = os.path.basename(nb_path).replace("ipynb", "py")
        nb_to_metaflow(
            nb_path, Path(os.path.join(flows_dir, flow_module_name)), silent=False
        )

In [160]:
generate_flows(Config(cfg_name="test/settings.ini"))

Skipping metaflow.ipynb - no steps
Skipping build.ipynb - no steps
Skipping parse_module.ipynb - no steps
Skipping scilint.ipynb - no steps
Skipping export_named.ipynb - no steps
Skipping packaging.ipynb - no steps
Skipping params.ipynb - no steps
Skipping index.ipynb - no export module
Wrote TestExportFlow to: test_export.py
Wrote TestModuleFlow to: test_module.py


In [161]:
# export


@call_parse
def sciflow_generate():
    generate_flows(Config())